In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
data = pd.read_csv('Churn_Modelling.csv')
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [3]:
## preprocessing the data
### drop inrrelevant columns
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [4]:
## Encode categorical columns
## Converts: Male → 1, Female → 0
## Why needed -> ANN cannot understand text, only numbers.

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender']) 


In [5]:
## one hot encoding for Geography column
## Why needed -> Geography has more than 2 categories, so One-Hot Encoding is correct.

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
geo_encoded = ohe.fit_transform(data[['Geography']])

feature_names=ohe.get_feature_names_out(['Geography'])


feature_names.tolist()

['Geography_France', 'Geography_Germany', 'Geography_Spain']

In [6]:
geo_encoded_df=pd.DataFrame(geo_encoded, columns=feature_names)
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [7]:
## combine the encoded columns with the original dataframe

## What this cell does -> Removes original text column and Concatenate encoded columns

## Why needed -> We already replaced it with numeric columns.

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [8]:
## Save the encoders and scaler 

with open('label_encoder_gender.pkl', 'wb') as f:
    pickle.dump(label_encoder_gender, f)
    
with open('onehot_encoder_geography.pkl', 'wb') as f:
    pickle.dump(ohe, f)

In [9]:
## divide the data into dependant and independent features

data.head()

## i will keep 'Exited' column as the target variable and rest all as independent features
X=data.drop('Exited', axis=1)
y=data['Exited']

## Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)   



In [10]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [11]:
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

## Implemenation ANN

In [12]:
import sys
sys.executable

'C:\\Users\\GauravKumar\\AppData\\Local\\Programs\\Python\\Python310\\python.exe'

In [13]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.models import Sequential
print("keras ok")
from tensorflow.keras.layers import Dense
print("layers ok")
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime




2.15.0
keras ok
layers ok


In [14]:
X_train.shape


## 8000 -> Rows and 12 -> columns

(8000, 12)

In [15]:
X_train.shape[1]

12

In [16]:
X_train[:5]
for i, value in enumerate(X_train[1]):
    print(f"Feature {i}: {value}")


Feature 0: -0.20389776790135397
Feature 1: 0.9132475520046849
Feature 2: 0.29493846545136665
Feature 3: -0.34836910282030104
Feature 4: 0.6968376547320528
Feature 5: 0.8084361535388619
Feature 6: 0.6492026709982579
Feature 7: 0.9748169889124099
Feature 8: 1.6612541021719276
Feature 9: -0.9985011233143956
Feature 10: 1.7257231348452826
Feature 11: -0.5763880180721888


In [17]:
## Build the ANN model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), ## 1st hidden layer connected to input layer
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
import tensorflow

opt = tensorflow.keras.optimizers.Adam(learning_rate=0.001)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [20]:
## compile the model
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [21]:
## set up the temsorboard 
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [22]:
## Set up EarlyStopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [23]:
## Train the model
history = model.fit(
    X_train, y_train,validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[tensorboard_callback, early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 2s 4ms/step - loss: 0.4480 - accuracy: 0.8059 - val_loss: 0.3934 - val_accuracy: 0.8300
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3836 - accuracy: 0.8435 - val_loss: 0.3586 - val_accuracy: 0.8525
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3550 - accuracy: 0.8524 - val_loss: 0.3446 - val_accuracy: 0.8595
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3446 - accuracy: 0.8555 - val_loss: 0.3451 - val_accuracy: 0.8620
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3378 - accuracy: 0.8621 - val_loss: 0.3481 - val_accuracy: 0.8545
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3346 - accuracy: 0.8593 - val_loss: 0.3403 - val_accuracy: 0.8625
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3305 - accuracy: 0.8622 - val_loss: 0.3422 - val_accuracy: 0.85

In [24]:
model.save('ann_model_gaurav.h5')

C:\Users\GauravKumar\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [25]:
## load Tensorboard Extension 
%load_ext tensorboard

In [52]:
!kill 14688
%tensorboard --logdir logs/fit --port 6006


'kill' is not recognized as an internal or external command,
operable program or batch file.


ERROR: Failed to launch TensorBoard (exited with 4294967295).
Contents of stderr:
2026-01-05 11:35:16.413288: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-05 11:35:22.283660: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
E0105 11:35:24.580364 45040 program.py:300] TensorBoard could not bind to port 6006, it was already in use
ERROR: TensorBoard could not bind to port 6006, it was already in use

In [26]:
%tensorboard --logdir logs/fit

In [ ]:
### Load the pickle File
